# Building an AI-Powered Document Retrieval System with Docling and Granite

*Using IBM Granite Models*

## Recipe Overview

Welcome to this Granite recipe, in this recipe, you'll learn to harness the power of advanced tools to build AI-powered document retrieval systems. It will guide you through:

- **Document Processing:** Learn to handle documents from various sources, parse and transform them into usable formats, and store them in vector databases using Docling.
- **Retrieval-Augmented Generation (RAG):** Understand how to connect large language models (LLMs) like Granite with external knowledge bases to enhance query responses and generate valuable insights.
- **LangChain for Workflow Integration:** Discover how to use LangChain to streamline and orchestrate document processing and retrieval workflows, enabling seamless interaction between different components of the system.

This recipe leverages three cutting-edge technologies:

1. **[Docling](https://docling-project.github.io/docling/):** An open-source toolkit for parsing and converting documents.
2. **[Granite](https://www.ibm.com/granite/docs/models/granite/):** A state-of-the-art LLM available via an [API](https://www.ibm.com/topics/api) through Replicate, providing robust natural language capabilities.
3. **[LangChain](https://github.com/langchain-ai/langchain):** A powerful framework for building applications powered by language models, designed to simplify complex workflows and integrate external tools seamlessly.

By the end of this recipe, you will:
- Gain proficiency in document processing and chunking.
- Integrate vector databases to enhance retrieval capabilities.
- Utilize RAG to perform efficient and accurate data retrieval for real-world applications.

This recipe is designed for AI developers, researchers, and enthusiasts looking to enhance their knowledge of document management and advanced NLP techniques.



## Prerequisites

- Familiarity with Python programming.
- Basic understanding of large language models and natural language processing concepts.


## Step 1: Setting up the environment

Install dependencies.

In [1]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install git+https://github.com/ibm-granite-community/utils \
    transformers \
    langchain_classic \
    langchain_core \
    langchain_huggingface sentence_transformers \
    langchain_milvus 'pymilvus[milvus_lite]' \
    docling \
    'langchain_replicate @ git+https://github.com/ibm-granite-community/langchain-replicate.git'
! echo "::endgroup::"

::group::Install Dependencies
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 89.9 MB/s eta 0:00:00
Using Python 3.12.12 environment at: /usr
Resolved 124 packages in 5.98s
Prepared 38 packages in 3.05s
Uninstalled 2 packages in 8ms
Installed 38 packages in 111ms
 + colorlog==6.10.1
 + docling==2.66.0
 + docling-core==2.57.0
 + docling-ibm-models==3.10.3
 + docling-parse==4.7.2
 + faker==40.1.0
 + filetype==1.2.0
 + ibm-granite-community-utils==0.1.dev126 (from git+https://github.com/ibm-granite-community/utils@f602c014defb9560be93ab52ef9fa9dbbd4d3eae)
 + jsonlines==4.0.0
 + jsonref==1.1.0
 + langchain-classic==1.0.1
 - langchain-core==1.2.1
 + langchain-core==1.2.6
 + langchain-huggingface==1.2.0
 + langchain-milvus==0.3.2
 + langchain-replicate==0.1.dev26 (from git+https://github.com/ibm-granite-community/langchain-replicate.git@200c6f94a8c3bb59afc5dda0dfd88490cd5ba952)
 + langchain-text-splitters==1.1.0
 + latex2mathml==3.78.1
 + marko==2.2.1
 + milvus-lite==2.5.1
 + mpire=

## Step 2: Selecting System Components

### Choose your Embeddings Model

Specify the model to use for generating embedding vectors from text. Here we will be using one of the new [Granite Embeddings models](https://huggingface.co/collections/ibm-granite/granite-embedding-models-6750b30c802c1926a35550bb)

To use a model from another provider, replace this code cell with one from [this Embeddings Model recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer

embeddings_model_path = "ibm-granite/granite-embedding-30m-english"
embeddings_model = HuggingFaceEmbeddings(
    model_name=embeddings_model_path,
)
embeddings_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_path)

### Use the Granite model

Select a Granite model from the [`ibm-granite`](https://replicate.com/ibm-granite) org on Replicate. Here we use the Replicate Langchain client to connect to the model.

To get set up with Replicate, see [Getting Started with Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb).

To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [7]:
from langchain_replicate import ChatReplicate
from ibm_granite_community.notebook_utils import get_env_var

model_path = "ibm-granite/granite-4.0-h-small"
model = ChatReplicate(
    model=model_path,
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),
    model_kwargs={
        "max_tokens": 1000, # Set the maximum number of tokens to generate as output.
        "min_tokens": 100, # Set the minimum number of tokens to generate as output.
    },
)

REPLICATE_API_TOKEN loaded from Google Colab secret.


Now that we have the model downloaded, let's try asking it a question

In [9]:
from langchain_core.prompts import ChatPromptTemplate

query = "Buggati or Maclaren"

# Create a Granite prompt for question-answering
prompt_template = ChatPromptTemplate.from_template(template="{input}")

chain = prompt_template | model

output = chain.invoke({"input": query})

print(output.text)

Both Bugatti and McLaren are renowned for their high-performance sports cars, but they cater to slightly different markets and have distinct characteristics.

Bugatti is known for its ultra-luxury, high-performance vehicles. The most famous model, the Bugatti Veyron, was one of the fastest and most expensive cars in the world when it was introduced. The current model, the Bugatti Chiron, continues this tradition with a quad-turbocharged 8.0-liter W16 engine that produces 1500 horsepower. Bugatti cars are known for their top speed, luxury, and exclusivity.

McLaren, on the other hand, is a British manufacturer known for its high-performance sports cars and involvement in Formula 1. McLaren cars are often lighter and more agile than their competitors, thanks to extensive use of carbon fiber in their construction. The McLaren 720S, for example, is a mid-engine sports car with a twin-turbocharged 4.0-liter V8 engine that produces 710 horsepower. McLaren cars are often praised for their han

Now, I know that UFC 310 happened in 2024, and this does not seem to be the right Pantoja. The model doesn't seem to know the answer but at least understands that this matchup did not occur. Let's see if it has some specific UFC rules info.

In [10]:
query1 = "How much weight allowance is allowed in non championship fights in the UFC?"

output = chain.invoke({"input": query1})

print(output.text)

In the UFC, for non-championship fights, the weight allowance is typically 1 pound. This means that fighters are allowed to weigh up to 1 pound over the specified weight limit for their respective weight class on the official weigh-ins. However, if a fighter misses weight, they may be subject to penalties such as a percentage deduction from their fight purse going to their opponent, or in some cases, the fight may be cancelled. It's always best to check the specific rules and regulations for each event as they can sometimes vary.


Based on the official UFC rules, this is also incorrect. Let's try getting some documents that contains this information for the model.

### Choose your Vector Database

Specify the database to use for storing and retrieving embedding vectors.

To connect to a vector database other than Milvus, replace this code cell with one from [this Vector Store recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [ ]:
from langchain_milvus import Milvus
import tempfile

db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
print(f"The vector database will be saved to {db_file}")

vector_db = Milvus(
    embedding_function=embeddings_model,
    connection_args={"uri": db_file},
    auto_id=True,
    enable_dynamic_field=True,
    index_params={"index_type": "AUTOINDEX"},
)

## Step 3: Building the Vector Database

In this example, from a set of source documents, we use [Docling](https://docling-project.github.io/docling/) to convert the documents into text and then split the text into chunks, derive embedding vectors using the embedding model, and load it into the vector database. Creating this vector database will allow us to easily search across our documents, enabling us to use RAG.

### Use Docling to download the documents, convert to text, and split into chunks

Here we have found a website that gives us information on UFC 310, as well as a PDF of the official UFC rules. Below, we will see that Docling can both convert and chunk the two documents.

In [ ]:
# Docling imports
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.labels import DocItemLabel
from langchain_core.documents import Document

# Here are our documents, feel free to add more documents in formats that Docling supports
sources = [
    "https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura",
    "https://media.ufc.tv/discover-ufc/Unified_Rules_MMA.pdf",
]

converter = DocumentConverter()

# Convert and chunk out documents
doc_id = 0
texts: list[Document] = [
    Document(page_content=chunk.text, metadata={"doc_id": (doc_id:=doc_id+1), "source": source})
    for source in sources
    for chunk in HybridChunker(tokenizer=embeddings_tokenizer).chunk(converter.convert(source=source).document)
    if any(filter(lambda c: c.label in [DocItemLabel.TEXT, DocItemLabel.PARAGRAPH], iter(chunk.meta.doc_items)))
]

print(f"{len(texts)} document chunks created")

In [ ]:
# Print all created documents
for document in texts:
    print(f"Document ID: {document.metadata['doc_id']}")
    print(f"Source: {document.metadata['source']}")
    print(f"Content:\n{document.page_content}")
    print("=" * 80)  # Separator for clarity

### Populate the vector database

NOTE: Population of the vector database may take over a minute depending on your embedding model and service.

In [ ]:
ids = vector_db.add_documents(texts)
print(f"{len(ids)} documents added to the vector database")

## Step 4: RAG with Granite

Now that we have succesfully converted our documents and vectorized them, we can set up out RAG pipeline.

### Retrieve relevant chunks



Here we will test the as_retriever method to search through our newly created vector database for chunks that are relevant to our original query



In [ ]:
retriever = vector_db.as_retriever()

docs = retriever.invoke(query)
print(docs)

Looks like it pulled some chunks that would have the information we are looking for. Let's go ahead and contruct our RAG pipeline.

### Create the prompt for Granite

Next, we construct the prompt pipeline. This creates the prompt which holds the retrieved chunks from out previous search and feeds this to the model as context for answering our question.

In [ ]:
from ibm_granite_community.langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

# Assemble the retrieval-augmented generation chain
combine_docs_chain = create_stuff_documents_chain(
    llm=model,
    prompt=prompt_template,
)
rag_chain = create_retrieval_chain(
    retriever=vector_db.as_retriever(),
    combine_docs_chain=combine_docs_chain,
)

### Generate a retrieval-augmented response to a question

The pipeline uses the query to locate documents from the vector database and use them as context for the query.

In [ ]:
output = rag_chain.invoke({"input": query})

print(output['answer'])

Awesome! It looks like the model figured out our first question. Let's see if it figure out the rule we were looking for.

In [ ]:
output = rag_chain.invoke({"input": query1})

print(output['answer'])

Awesome! We can now see that we have created a pipeline that can successfully leverage knowledge from multiple document types for generation.

## Next Steps

- Explore advanced RAG workflows for other industries
- Experiment with other document types and larger datasets.
- Optimize prompt engineering for better Granite responses.

Thank you for using this recipe!